In [783]:
#Importo las librerías a usar
import pandas as pd
import numpy as np

# ETL datasets asociados a cambios climaticos

-- Nota recomiendo para el pipelines solo automatizar para ese primer csvla eliminación de Message y Start_Date dado que se repite la información el resto de los datos a mi parecer están bien correlacionados

In [784]:
#Traigo los archivos a evaluar
df_calidadDelAire = pd.read_csv("Taxis-NewYork\Datasets\Air_Quality.csv")
df_energia = pd.read_csv("Taxis-NewYork\Datasets\energy.csv")
df_temperatura = pd.read_csv(r'C:\Users\Patricia\Desktop\Proyecto grupal\Taxis-NewYork\Datasets\archive (3).csv', sep=';', encoding='latin-1')
df_cambio_climatico = pd.read_csv('Taxis-NewYork\Datasets\climate.csv')




C:\Users\Patricia\AppData\Local\Temp\ipykernel_20336\2561296701.py:4: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temperatura = pd.read_csv(r'C:\Users\Patricia\Desktop\Proyecto grupal\Taxis-NewYork\Datasets\archive (3).csv', sep=';', encoding='latin-1')


In [785]:
df_calidadDelAire.info() #Evaluo la informacion existente

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16122 entries, 0 to 16121
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unique ID       16122 non-null  int64  
 1   Indicator ID    16122 non-null  int64  
 2   Name            16122 non-null  object 
 3   Measure         16122 non-null  object 
 4   Measure Info    16122 non-null  object 
 5   Geo Type Name   16122 non-null  object 
 6   Geo Join ID     16122 non-null  int64  
 7   Geo Place Name  16122 non-null  object 
 8   Time Period     16122 non-null  object 
 9   Start_Date      16122 non-null  object 
 10  Data Value      16122 non-null  float64
 11  Message         0 non-null      float64
dtypes: float64(2), int64(3), object(7)
memory usage: 1.5+ MB


In [786]:
#El total de entradas de la ultima columna están vacíos, el resto cuenta con los datos
df_calidadDelAire.isnull().sum()
#df_calidadDelAire.shape = (16122, 12) tenemos 16.122 filas y 12 columnas

Unique ID             0
Indicator ID          0
Name                  0
Measure               0
Measure Info          0
Geo Type Name         0
Geo Join ID           0
Geo Place Name        0
Time Period           0
Start_Date            0
Data Value            0
Message           16122
dtype: int64

In [787]:
#Nota: Chicos como les comenté anteriormente el problema de las fechas entonces solventé asi:

# 1.- Genero una funcion que detecte expresiones regulares para extraer solo la fecha y anexarla a otra columna llamada Final_Date
# 2.- Genero la columna de estaciones, sin embargo me encuentro con el problema de la fecha de inicio
# 3.- Por eso notarán que genero mas funciones para lograr extraer directamente el año

import re #Traigo la librería para buscar expresiones regulares

# Función para convertir el formato incorrecto del año a formato correcto (ejemplo: -09 a 2009)
def convert_year_format(year_str):
    match = re.search(r'\d{4}-(\d{2})', year_str) #Es una función de "re" que busca epresiones regulares
    if match:
        year = match.group(1)
        corrected_year = '20' + year
        return re.sub(r'-(\d{2})', '-' + corrected_year, year_str)
    else:
        return year_str

# Aplicamos la función a la columna 'Time Period' y almacenamos los resultados en la nueva columna 'Final_Date'
df_calidadDelAire['Final_Date'] = df_calidadDelAire['Time Period'].apply(convert_year_format)

# Función para extraer la estación del año y dejarla en una nueva columna 'Stations'
def extract_season(year_str):
    match = re.search(r'(Summer|Winter)', year_str)
    if match:
        return match.group(0)
    else:
        return 'Annual'  # Si no se encuentra la estación, asumimos que es un dato anual

# Aplicamos la función a la columna 'Time Period' y almacenamos los resultados en la nueva columna 'Stations'
df_calidadDelAire['Stations'] = df_calidadDelAire['Time Period'].apply(extract_season)



In [788]:
# Utilizar str.extract con expresión regular para obtener el último año en el formato año
df_calidadDelAire['Final_Date'] = df_calidadDelAire['Final_Date'].str.extract(r'(\d{4})')


In [789]:
#Extraigo el año de inicio anual
df_calidadDelAire['Start_Date'] = df_calidadDelAire['Start_Date'].str.extract(r'(\d{4})')



In [790]:
df_calidadDelAire.head(3)

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message,Final_Date,Stations
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,2013,34.64,NaN,2013,Summer
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,2014,33.22,NaN,2014,Summer
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,2013,31.25,NaN,2013,Summer


In [791]:
#Elimino la columna que no tiene ningun dato representativo
df_calidadDelAire.drop(columns=['Message', 'Time Period'], inplace=True)

In [792]:
#Verifico que los datos estén quedando bien
df_calidadDelAire.head(5)

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Start_Date,Data Value,Final_Date,Stations
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),2013,34.64,2013,Summer
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),2014,33.22,2014,Summer
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,2013,31.25,2013,Summer
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,2014,31.15,2014,Summer
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),2008,5.89,2008,Winter


In [793]:
# Exportar el DataFrame a un archivo Excel para verificar los datos
#df_calidadDelAire.to_excel('borrar.xlsx', index=False)


In [ ]:
df_calidadDelAire.to_csv('Calidad del aire.csv', index=False) # Genero los datos a un csv

In [794]:
# Inicio con el segundo dataset sin embargo noto que contiene datos de muchos países y años por lo cual utilizo un filtro

df_filtrado = df_energia.loc[(df_energia['Country'] == 'United States') & (df_energia['Year'] >= 1980)]

In [795]:
df_filtrado.isnull().sum() #Notese que pese a que no salen valores nulos en la columna CO2_emission hay valores 0
#Son 41 datos con numero 0, no están asociado a uso de energías que generen emision de CO2
# df_filtrado.shape --> (240, 11)

Unnamed: 0                     0
Country                        0
Energy_type                    0
Year                           0
Energy_consumption             0
Energy_production              0
GDP                            0
Population                     0
Energy_intensity_per_capita    0
Energy_intensity_by_GDP        0
CO2_emission                   0
dtype: int64

In [796]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240 entries, 1320 to 55379
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   240 non-null    int64  
 1   Country                      240 non-null    object 
 2   Energy_type                  240 non-null    object 
 3   Year                         240 non-null    int64  
 4   Energy_consumption           240 non-null    float64
 5   Energy_production            240 non-null    float64
 6   GDP                          240 non-null    float64
 7   Population                   240 non-null    float64
 8   Energy_intensity_per_capita  240 non-null    float64
 9   Energy_intensity_by_GDP      240 non-null    float64
 10  CO2_emission                 240 non-null    float64
dtypes: float64(7), int64(2), object(2)
memory usage: 22.5+ KB


In [797]:
df_filtrado.head(3)

,Unnamed: 0,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
1320,1320,United States,all_energy_types,1980,78.021113,67.146595,7080.75,227119.0,343.525258,11.018764,4946.627130
1321,1321,United States,coal,1980,15.422809,18.597725,7080.75,227119.0,343.525258,11.018764,1409.790188
1322,1322,United States,natural_gas,1980,20.235459,19.907600,7080.75,227119.0,343.525258,11.018764,1081.593377


In [798]:
#El Energy_intensity_by_GDP es una medida  que sirve para correlacionar el crecimiento del PIB con el gasto de energía de un país a menor consumo 
#de GDP mejor porque significa mejor uso de recursos energeticos


# df_filtrado['Energy_intensity_by_GDP'].min() #Se corresponde con el año 2019

In [ ]:
df_filtrado.to_csv('Energía.csv', index=False) # Genero los datos a un csv

In [799]:
#Genero las nuevas columnas del datasets

nombre_columnas = ['time', 'temperature_2m (°C)', 'relativehumidity_2m (%)', 'apparent_temperature (°C)',
                                 'rain (mm)', 'is_day ()', 'latitude', 'longitude', 'elevation', 'utc_offset_seconds',
                                 'timezone', 'timezone_abbreviation']

In [800]:
# Asignar los nuevos nombres de columnas al DataFrame original
df_temperatura.columns = nombre_columnas

In [801]:
df_temperatura.drop(columns=['latitude', 'longitude', 'elevation', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation'], inplace=True)



In [802]:
df_temperatura.drop(0, inplace= True)

In [803]:
df_temperatura.reset_index()

,index,time,temperature_2m (°C),relativehumidity_2m (%),apparent_temperature (°C),rain (mm),is_day ()
0,1,2010-01-01T00:00,-2.6,88,-7.7,0.00,0
1,2,2010-01-01T01:00,-2.7,88,-7.7,0.00,0
2,3,2010-01-01T02:00,-2.7,88,-7.7,0.00,0
3,4,2010-01-01T03:00,-2.7,88,-7.6,0.00,0
4,5,2010-01-01T04:00,-2.7,88,-7.5,0.00,0
...,...,...,...,...,...,...,...
118939,118940,2023-07-27T19:00,NaN,NaN,NaN,NaN,1
118940,118941,2023-07-27T20:00,NaN,NaN,NaN,NaN,0
118941,118942,2023-07-27T21:00,NaN,NaN,NaN,NaN,0
118942,118943,2023-07-27T22:00,NaN,NaN,NaN,NaN,0


In [804]:
df_temperatura.replace('NaN', np.nan, inplace=True)  # Reemplaza 'NaN' por np.nan

In [805]:
df_temperatura.dropna(inplace=True)

In [806]:
df_temperatura.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118896 entries, 1 to 118896
Data columns (total 6 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   time                       118896 non-null  object
 1   temperature_2m (°C)        118896 non-null  object
 2   relativehumidity_2m (%)    118896 non-null  object
 3   apparent_temperature (°C)  118896 non-null  object
 4   rain (mm)                  118896 non-null  object
 5   is_day ()                  118896 non-null  object
dtypes: object(6)
memory usage: 6.3+ MB


In [807]:
#Evalúo cantidad de valores nulos previo a la transformacion de datos
df_temperatura.isnull().sum()

time                         0
temperature_2m (°C)          0
relativehumidity_2m (%)      0
apparent_temperature (°C)    0
rain (mm)                    0
is_day ()                    0
dtype: int64

In [808]:
# Transformo los datos a tipo de dato fecha
df_temperatura['time'] = pd.to_datetime(df_temperatura['time'], errors='coerce')


In [809]:
# Me cercioro que no alteré nada en la columna de time
# df_temperatura['time'].isnull().sum() --> 0


In [810]:
# Elimino la hora del formato
df_temperatura['time'] = df_temperatura['time'].dt.date


In [811]:
#Elimino los valores nulos
df_temperatura.dropna(inplace=True)

In [812]:
df_temperatura.head(5) #Verifico el resultado

,time,temperature_2m (°C),relativehumidity_2m (%),apparent_temperature (°C),rain (mm),is_day ()
1,2010-01-01,-2.6,88,-7.7,0.00,0
2,2010-01-01,-2.7,88,-7.7,0.00,0
3,2010-01-01,-2.7,88,-7.7,0.00,0
4,2010-01-01,-2.7,88,-7.6,0.00,0
5,2010-01-01,-2.7,88,-7.5,0.00,0


In [813]:
#Verifico que no existan nulos.
df_temperatura.isnull().sum()

time                         0
temperature_2m (°C)          0
relativehumidity_2m (%)      0
apparent_temperature (°C)    0
rain (mm)                    0
is_day ()                    0
dtype: int64

In [ ]:
df_temperatura.to_csv('Temperatura.csv', index=False) # Genero los datos a un csv

In [814]:
df_cambio_climatico.head(3)

,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C)
0,2000-01-01,-2.0,1.6,-4.9
1,2000-01-02,-0.2,3.4,-2.2
2,2000-01-03,0.9,2.3,-0.6


In [815]:
df_cambio_climatico.columns

Index(['time', 'temperature_2m_mean (°C)', 'temperature_2m_max (°C)',
       'temperature_2m_min (°C)'],
      dtype='object')

In [816]:
#Verifico que no existan nulos
df_cambio_climatico.isnull().sum()

time                        0
temperature_2m_mean (°C)    0
temperature_2m_max (°C)     0
temperature_2m_min (°C)     0
dtype: int64

In [817]:
df_cambio_climatico.info() #Reviso tipo de datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8614 entries, 0 to 8613
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   time                      8614 non-null   object 
 1   temperature_2m_mean (°C)  8614 non-null   float64
 2   temperature_2m_max (°C)   8614 non-null   float64
 3   temperature_2m_min (°C)   8614 non-null   float64
dtypes: float64(3), object(1)
memory usage: 269.3+ KB


In [818]:
#Transformo la fecha de tipo de dato objeto o string a fecha
df_cambio_climatico ['time'] = pd.to_datetime(df_cambio_climatico['time'], errors='coerce')

In [819]:
#Verifico que todo está bien y reviso que no se hayan quedado nulos otra vez y dan 0 nulos
df_cambio_climatico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8614 entries, 0 to 8613
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   time                      8614 non-null   datetime64[ns]
 1   temperature_2m_mean (°C)  8614 non-null   float64       
 2   temperature_2m_max (°C)   8614 non-null   float64       
 3   temperature_2m_min (°C)   8614 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 269.3 KB


In [ ]:
df_cambio_climatico.to_csv('Cambio climatico.csv', index=False) # Genero los datos a un csv